<a href="https://colab.research.google.com/github/kirsten-fung/project_voting/blob/main/voting_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
! git clone https://github.com/kirsten-fung/project_voting

fatal: destination path 'project_voting' already exists and is not an empty directory.


In [40]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [41]:
# read in csv
nhgis = pd.read_csv('/content/project_voting/cleandf.csv')
va = pd.read_csv('/content/project_voting/data/voting_VA.csv')

In [42]:
# only look at virginia data in the country dataset
df = nhgis[nhgis['State Name'] == 'Virginia']
df.head()

,GIS Join Match Code,Data File Year,State Postal Abbreviation,State Name,State Code,County Name,County Code,Geographic Identifier,Area Name,Total,...,Built 2000 to 2004,Built 1990 to 1999,Built 1980 to 1989,Built 1970 to 1979,Built 1960 to 1969,Built 1950 to 1959,Built 1940 to 1949,Built 1939 or earlier,Median gross rent,Median value (dollars)
2817,G5100010,2006-2010,VA,Virginia,51,Accomack County,1,05000US51001,"Accomack County, Virginia",34066,...,991,3212,3026,3261,2148,2025,899,4965,709.0,149800
2818,G5100030,2006-2010,VA,Virginia,51,Albemarle County,3,05000US51003,"Albemarle County, Virginia",96633,...,5830,10158,7375,6827,3092,2070,816,3053,1031.0,349800
2819,G5100050,2006-2010,VA,Virginia,51,Alleghany County,5,05000US51005,"Alleghany County, Virginia",16406,...,196,944,736,1478,1182,844,687,1849,504.0,104000
2820,G5100070,2006-2010,VA,Virginia,51,Amelia County,7,05000US51007,"Amelia County, Virginia",12517,...,739,1173,839,1010,483,317,301,322,672.0,189800
2821,G5100090,2006-2010,VA,Virginia,51,Amherst County,9,05000US51009,"Amherst County, Virginia",32315,...,704,2476,2409,2847,1599,1460,592,1454,582.0,142200


In [43]:
# group variables
df['Other'] = df['Some other race alone'] + df['Two or more races']
df['Male Higher Education'] = df["Male: Associate's degree"] + df["Male: Bachelor's degree"] + df["Male: Master's degree"] + df["Male: Doctorate degree"] + df["Male: Professional school degree"]
df['Female Higher Education'] = df["Female: Associate's degree"] + df["Female: Bachelor's degree"] + df["Female: Master's degree"] + df["Female: Doctorate degree"] + df["Female: Professional school degree"]
df['Male Highschool Equivalent'] = df['Male: High school graduate, GED, or alternative'] + df['Male: Some college, less than 1 year']
df['Female Highschool Equivalent'] = df['Female: High school graduate, GED, or alternative'] + df['Female: Some college, less than 1 year']

<ipython-input-43-845c701cb365>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Other'] = df['Some other race alone'] + df['Two or more races']
<ipython-input-43-845c701cb365>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Male Higher Education'] = df["Male: Associate's degree"] + df["Male: Bachelor's degree"] + df["Male: Master's degree"] + df["Male: Doctorate degree"] + df["Male: Professional school degree"]
<ipython-input-43-845c701cb365>:4: SettingWithCopyWarning: 
A value is trying to be set

In [44]:
# create new df for variables we want
df = df[['State Name','County Name','Male','Female','Median age: Male','Median age: Female','White alone','Black or African American alone','American Indian and Alaska Native alone','Asian alone','Native Hawaiian and Other Pacific Islander alone','Hispanic or Latino','Other','Male Higher Education','Female Higher Education','Male Highschool Equivalent','Female Highschool Equivalent','Median household income in the past 12 months (in 2010 inflation-adjusted dollars)']]
df.head()

,State Name,County Name,Male,Female,Median age: Male,Median age: Female,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Hispanic or Latino,Other,Male Higher Education,Female Higher Education,Male Highschool Equivalent,Female Highschool Equivalent,Median household income in the past 12 months (in 2010 inflation-adjusted dollars)
2817,Virginia,Accomack County,16707,17359,41.5,45.8,22639,9729,30,166,0,2732,1502,2622,3077,5080,5103,41372
2818,Virginia,Albemarle County,46330,50303,36.3,38.9,79493,9563,168,4146,28,4790,3235,16690,19230,6851,8238,64847
2819,Virginia,Alleghany County,8072,8334,43.2,46.0,15250,979,3,81,0,72,93,1049,1857,2735,2571,43160
2820,Virginia,Amelia County,6153,6364,40.6,42.7,9228,2954,31,47,18,292,239,727,847,1885,2062,50135
2821,Virginia,Amherst County,15347,16968,39.9,42.2,24762,6201,301,149,0,675,902,2077,2842,4189,5046,44757


In [47]:
# correlation matrix for PCA
X_country = df.loc[:, ['Male','Female','Median age: Male','Median age: Female','White alone','Black or African American alone','American Indian and Alaska Native alone','Asian alone','Native Hawaiian and Other Pacific Islander alone','Hispanic or Latino','Other','Male Higher Education','Female Higher Education','Male Highschool Equivalent','Female Highschool Equivalent','Median household income in the past 12 months (in 2010 inflation-adjusted dollars)'] ]
X_country.corr()

,Male,Female,Median age: Male,Median age: Female,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Hispanic or Latino,Other,Male Higher Education,Female Higher Education,Male Highschool Equivalent,Female Highschool Equivalent,Median household income in the past 12 months (in 2010 inflation-adjusted dollars)
Male,1.000000,0.999167,-0.266315,-0.283764,0.990295,0.760742,0.944193,0.902761,0.877359,0.931605,0.937869,0.964377,0.974902,0.929523,0.954476,0.444397
Female,0.999167,1.000000,-0.267448,-0.286538,0.989872,0.768926,0.941555,0.897345,0.869574,0.925150,0.931900,0.962098,0.973987,0.931568,0.958326,0.441165
Median age: Male,-0.266315,-0.267448,1.000000,0.908633,-0.240930,-0.366888,-0.201822,-0.137141,-0.308032,-0.209248,-0.235959,-0.197393,-0.207648,-0.278897,-0.272990,-0.109875
Median age: Female,-0.283764,-0.286538,0.908633,1.000000,-0.268707,-0.342245,-0.220597,-0.156119,-0.319306,-0.237396,-0.263413,-0.225266,-0.233731,-0.284612,-0.283122,-0.212155
White alone,0.990295,0.989872,-0.240930,-0.268707,1.000000,0.689999,0.923344,0.899483,0.845875,0.916538,0.919313,0.962898,0.973064,0.917688,0.946670,0.464507
Black or African American alone,0.760742,0.768926,-0.366888,-0.342245,0.689999,1.000000,0.675408,0.487342,0.752481,0.584601,0.630128,0.609107,0.643442,0.861213,0.842134,0.229897
American Indian and Alaska Native alone,0.944193,0.941555,-0.201822,-0.220597,0.923344,0.675408,1.000000,0.931874,0.801474,0.936371,0.920816,0.954501,0.953868,0.816327,0.853167,0.404108
Asian alone,0.902761,0.897345,-0.137141,-0.156119,0.899483,0.487342,0.931874,1.000000,0.743337,0.944887,0.912544,0.970581,0.959475,0.698391,0.751112,0.394594
Native Hawaiian and Other Pacific Islander alone,0.877359,0.869574,-0.308032,-0.319306,0.845875,0.752481,0.801474,0.743337,1.000000,0.826085,0.845171,0.808135,0.819479,0.857469,0.853926,0.385498
Hispanic or Latino,0.931605,0.925150,-0.209248,-0.237396,0.916538,0.584601,0.936371,0.944887,0.826085,1.000000,0.991607,0.960125,0.951878,0.769395,0.811778,0.458439
